In [7]:
from PIL import Image
import pandas as pd
import numpy as np

dataRep = '../data/'
datasetRep = '../data/dataset/'

unicodeData = pd.read_csv(dataRep+'unicode_translation.csv')
trainData = pd.read_csv(dataRep+'train.csv')
sampleSubData = pd.read_csv(dataRep+'sample_submission.csv')

In [2]:
import time, sys
from IPython.display import clear_output

def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

In [70]:
def makeSquareImage(im, fill_color=(0, 0, 0)):
    x, y = im.size
    size = max(x, y)
    new_im = Image.new('RGB', (size, size), fill_color)
    new_im.paste(im, (int((size - x) / 2), int((size - y) / 2)))
    return new_im

def convertImage(image, xpixel=1024, ypixel=1024, gray=False):
    #convert size
    image.thumbnail([xpixel, ypixel], Image.LANCZOS)
    image.convert('RGB')
    image = makeSquareImage(image, fill_color=(255,255,255))
    
    #convert color
    if gray:
        image.convert('L')
        
    return image


def createCaracDatabase(label):
    carac = []
    pos = []
    size = []
    if(not isinstance(label, float)):
        labelS = label.split()
        for i in range(0, len(labelS)//5, 5):
            carac.append(unicodeData[unicodeData['Unicode']=='U+0032']['char'].iloc[0])
            pos.append((int(labelS[i+1]),int(labelS[i+2])))
            size.append((int(labelS[i+3]),int(labelS[i+4])))
    
    return pd.DataFrame(list(zip(carac, pos, size)), columns=['caracter', 'position', 'size'])
    
    
def extractCaracterFromImage(image, pos, size, finalSize=(32,32), gray=True):
    box = (pos[0], pos[1], size[0], size[1])
    return convertImage(image.copy().crop(box), xpixel=finalSize[0], ypixel=finalSize[0], gray=gray)


def fromPageCreateCaracterDataset(image, label):
    caracDB = createCaracDatabase(label)
    imageCaracList = []
    caracList = []
    for i in range(0, len(caracDB)):
        caracList.append(caracDB.iloc[i]['caracter'])
        if (caracDB.iloc[i]['size'][0] > 0) and (caracDB.iloc[i]['size'][1] > 0):
            imageCaracList.append(np.asarray(extractCaracterFromImage(image, caracDB.iloc[i]['position'], caracDB.iloc[i]['size'])).copy())
            print(type(imageCaracList[-1]), imageCaracList[-1].size, imageCaracList[-1].shape, imageCaracList[-1].dtype)
        else:
            print('error')
    return caracList, imageCaracList


def createDatasetFirstNetwork(xpixel=1024, ypixel=1024, gray=False):
    nImage = trainData.shape[0]
    
    i=0
    print('Convert train image')
    print('\n')
    for idImage in trainData['image_id']:
        i+=1
        update_progress(float(i)/float(nImage))
        image = Image.open(dataRep+'train_images/'+idImage+'.jpg')
        image = convertImage(image,xpixel, ypixel, gray)
        image.save(datasetRep+'train/'+idImage+'.jpg')
        

def createDatasetSecondNetwork():
    nImage = trainData.shape[0]
    nImage = 30
    imageCaracList = []
    caracList = []
    
    i=0
    print('Convert train image')
    print('\n')
    for j in range(0, nImage):
        i+=1
        #update_progress(float(i)/float(nImage))
        
        idImage = trainData['image_id'].iloc[j]
        label = trainData['labels'].iloc[j]
        image = Image.open(dataRep+'train_images/'+idImage+'.jpg')
        tmp1, tmp2 = fromPageCreateCaracterDataset(image, label)
        caracList += tmp1
        imageCaracList += tmp2
        
    caracList = np.asarray(caracList)
    imageCaracList = np.asarray(imageCaracList)
    print(imageCaracList.shape)
    print(imageCaracList.dtype)
    print(imageCaracList[0].shape)
    print(imageCaracList[0].dtype)
    np.savez_compressed(datasetRep+'caracterClassification.npz', caracter = caracList, image = imageCaracList)

In [71]:
createDatasetSecondNetwork()

Convert train image


<class 'numpy.ndarray'> 1 () object
1
<class 'numpy.ndarray'> 1 () object
1
<class 'numpy.ndarray'> 1 () object
1
<class 'numpy.ndarray'> 1 () object
1
<class 'numpy.ndarray'> 1 () object
1
<class 'numpy.ndarray'> 1 () object
1
<class 'numpy.ndarray'> 1 () object
1
<class 'numpy.ndarray'> 1 () object
1
<class 'numpy.ndarray'> 1 () object
1
<class 'numpy.ndarray'> 1 () object
1
<class 'numpy.ndarray'> 1 () object
1
<class 'numpy.ndarray'> 1 () object
1
<class 'numpy.ndarray'> 1 () object
1
<class 'numpy.ndarray'> 1 () object
1
<class 'numpy.ndarray'> 1 () object
1
<class 'numpy.ndarray'> 1 () object
1
<class 'numpy.ndarray'> 1 () object
1
<class 'numpy.ndarray'> 1 () object
1
<class 'numpy.ndarray'> 1 () object
1
<class 'numpy.ndarray'> 1 () object
1
<class 'numpy.ndarray'> 1 () object
1
<class 'numpy.ndarray'> 1 () object
1
<class 'numpy.ndarray'> 1 () object
1
<class 'numpy.ndarray'> 1 () object
1
<class 'numpy.ndarray'> 1 () object
1
<class 'numpy.ndarray'> 1 ()

SystemError: tile cannot extend outside image

In [13]:
unicodeData[unicodeData['Unicode']=='U+0032']['char'].iloc[0]

'2'

In [15]:
unicodeData.iloc[2]

KeyError: 2